In [204]:
import numpy as np
import pandas as pd
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
import json
import random
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
lemmatizer = WordNetLemmatizer()
words= []# keyword to predict 
classes =[] # class (goodbye, hello,thank,...)
documents =[] # () tuple chứa classes and word
ignore_word =['?','!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
print(intents)
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word 
        w = nltk.word_tokenize(pattern)
        words.extend(w) # extend =>array, append => array 2 dimension
        # add document each w
        documents.append((w, intent['tag']) )
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print("words first \n",words)
print("documents \n",documents)
print("classes \n",classes)
# lemmatize each word lower, remove word duplicate
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_word]
print("words\n",words)
words = sorted(list(set(words)))
print("words after:\n",words)
# sorted classes
classes = sorted(list(set(classes)))
print("classes \n", classes)
# document = combination pattern and intent
print(len(documents),'documents')
print(len(classes),'classes',classes)
print(len(words),'unique lemmatize word',words)
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))
# create training data
training =[]
# create empty array
output_empty =[0]* len(classes)
# print(output_empty)
# set training , bag of word in senquence
for doc in documents:
    #set bag of word
    bag =[]
    # list word tokenize for in pattern
    pattern_words = doc[0]
#     print("pattern_words",pattern_words)
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        #print(bag)
    #output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    #print(output_row)
    output_row[classes.index(doc[1])]=1
    #print(output_row)
    
    training.append([bag, output_row])
print(training)
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("train data created!!!!")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape =(len(train_x[0]),),activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation ='softmax'))
print(model)
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
print(sgd)
model.compile(loss ='categorical_crossentropy',optimizer =sgd, metrics =['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x),np.array(train_y),epochs =200, batch_size =5, verbose =1)
model.save('chatbot_model.h5',hist)
print("model created")

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

Epoch 1/200
47/47 [==============================] - 1s 15ms/sample - loss: 2.2798 - acc: 0.1702
Epoch 2/200
47/47 [==============================] - 0s 471us/sample - loss: 2.1579 - acc: 0.1489
Epoch 3/200
47/47 [==============================] - 0s 488us/sample - loss: 2.1440 - acc: 0.1064
Epoch 4/200
47/47 [==============================] - 0s 531us/sample - loss: 2.0092 - acc: 0.2766
Epoch 5/200
47/47 [==============================] - 0s 509us/sample - loss: 1.9437 - acc: 0.4255
Epoch 6/200
47/47 [==============================] - 0s 509us/sample - loss: 1.7580 - acc: 0.4894
Epoch 7/200
47/47 [==============================] - 0s 509us/sample - loss: 1.6716 - acc: 0.5319
Epoch 8/200
47/47 [==============================] - 0s 530us/sample - loss: 1.6205 - acc: 0.5957
Epoch 9/200
47/47 [==============================] - 0s 552us/sample - loss: 1.2960 - acc: 0.7021
Epoch 10/200
47/47 [==============================] - 0s 594us/sample - loss: 1.3774 - acc: 0.5745
Epoch 11/200
47/47 [

47/47 [==============================] - 0s 615us/sample - loss: 0.1077 - acc: 0.9787
Epoch 84/200
47/47 [==============================] - 0s 615us/sample - loss: 0.0699 - acc: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 596us/sample - loss: 0.0470 - acc: 1.0000
Epoch 86/200
47/47 [==============================] - 0s 467us/sample - loss: 0.0487 - acc: 0.9787
Epoch 87/200
47/47 [==============================] - 0s 488us/sample - loss: 0.0229 - acc: 1.0000
Epoch 88/200
47/47 [==============================] - 0s 479us/sample - loss: 0.1075 - acc: 0.9574
Epoch 89/200
47/47 [==============================] - 0s 488us/sample - loss: 0.0662 - acc: 0.9787
Epoch 90/200
47/47 [==============================] - 0s 467us/sample - loss: 0.0271 - acc: 1.0000
Epoch 91/200
47/47 [==============================] - 0s 488us/sample - loss: 0.0230 - acc: 1.0000
Epoch 92/200
47/47 [==============================] - 0s 509us/sample - loss: 0.1541 - acc: 0.9574
Epoch 93/200
47/47 [===

47/47 [==============================] - 0s 488us/sample - loss: 0.0064 - acc: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 616us/sample - loss: 0.0138 - acc: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 640us/sample - loss: 0.0159 - acc: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 552us/sample - loss: 0.0201 - acc: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 511us/sample - loss: 0.0152 - acc: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 525us/sample - loss: 0.0143 - acc: 1.0000
Epoch 171/200
47/47 [==============================] - 0s 552us/sample - loss: 0.0471 - acc: 0.9787
Epoch 172/200
47/47 [==============================] - 0s 976us/sample - loss: 0.0482 - acc: 0.9574
Epoch 173/200
47/47 [==============================] - 0s 679us/sample - loss: 0.0324 - acc: 0.9787
Epoch 174/200
47/47 [==============================] - 0s 594us/sample - loss: 0.0409 - acc: 1.0000
Epoch 175/200


In [149]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import json
from tensorflow.keras.models import load_model
import random
model = load_model('chatbot_model.h5')
print(model)
intents = json.loads(open('intents.json').read())
print(intents)
words   = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
print("words",words)
print("classes",classes)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [213]:
def clean_up_sentence( sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words =[lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
def bow(sentence, words, show_details = True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if s == w:
                # assign 1 if current word is in the vocabulary position
                bag[i] =1
                if show_details :
                    print("found in bag %s"%w)
    return (np.array(bag))
def predict_class( sentence, model):
    # filter out predictions below a threshold
    p = bow( sentence, words, show_details = True)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD =0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorted probability strength
    results.sort(key = lambda x: x[1], reverse =True)
    return_list =[]
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1]) })
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if (i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res  = getResponse(ints,intents)
    return res

In [216]:
# Creating GUI with Tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0","end-1c").strip()
    EntryBox.delete("0.0",END)
    if msg !="":
        ChatLog.config(state = NORMAL)
        ChatLog.insert(END,"You: "+msg+"\n\n")
        ChatLog.config(foreground ='#442265',font =("Vedana",12))
        res = chatbot_response(msg)
        ChatLog.insert(END, "BOT: "+res+'\n\n')
        ChatLog.config(state = DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title('Hello Truc')
base.geometry("400x500")
base.resizable(width =False, height =False)
# create chatlog window
ChatLog = Text(base, bd =0,bg ='white', height ='8',width ='50', font='Arial')
ChatLog.config(state = DISABLED)
# bind scrollbar chat window
scrollbar = Scrollbar(base, command = ChatLog.yview, cursor ='heart')
ChatLog['yscrollcommand'] = scrollbar.set
#  create button to send msg
SendButton =Button(base, font =("Vedana",12,'bold'), text ='Send', width =12, height =5,bd =0, bg ='#32de97',activebackground ='#3c9d9b',command =send,fg='#ffffff')
#  create box to enter message
EntryBox = Text(base, bd =0, bg ='white',height =5, width ='29',font ='Arial')
#  Place enter component in screen
scrollbar.place(x=376, y=6, height =386)
ChatLog.place (x=6,y=6, width= 370, height =386)
EntryBox.place(x=128,y =401, height =90, width =265)
SendButton.place(x=6, y=401, height =90)
base.mainloop()

found in bag hello
found in bag how
found in bag are
found in bag you
